# Домашнее задание (объединения в pandas)
  
## Задание 1
  
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [67]:
import pandas as pd

In [68]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [69]:
users = ratings.groupby('userId').count()
users = users[users.rating >= 100]
users.head()

,movieId,rating,timestamp
userId,,,
4,204,204,204
5,100,100,100
8,116,116,116
15,1700,1700,1700
17,363,363,363


In [70]:
time = ratings.groupby('userId').agg(['min', 'max'])['timestamp']
time.head()

,min,max
userId,,
1,1260759108,1260759205
2,835355395,835356246
3,1298861589,1298932787
4,949778714,949982274
5,1163373044,1163375145


In [74]:
lifetime['lifetime'] = lifetime.apply(lambda x: x['max'] - x['min'], axis=1)
lifetime

,movieId,rating,timestamp,min,max,lifetime
userId,,,,,,
4,204,204,204,949778714,949982274,203560
5,100,100,100,1163373044,1163375145,2101
8,116,116,116,1154389340,1154474527,85187
15,1700,1700,1700,997937239,1469330735,471393496
17,363,363,363,1127468587,1127476640,8053
...,...,...,...,...,...,...
656,128,128,128,986240991,986244044,3053
659,142,142,142,834598040,866207451,31609411
664,519,519,519,1343731283,1441911722,98180439


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:  
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [75]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [76]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [77]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [78]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [88]:
# три типа выручки с адресами
full = (rzd
            .merge(auto, how='outer')
            .merge(air, how='outer')
            .merge(client_base, how='outer')
       )
full.head()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1


In [92]:
# три типа выручки без адресов

revenue_full = full.drop(['address'], axis=1)
revenue_full.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

*лучше добавить отдельной таблицей "ИД визита - широта - долгота"*
  
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

*Координаты легко сделать числами, представив их в формате "Только градусы с десятичной дробной частью (54.97158, 73.38318)"*


* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

*тут не понял вопроса*


* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

*По координатам можно определять расстояния между какими либо действиями или место, в котором человек предпочитает совершать действия (к примеру, рекламу смотрел на работе, а покупку оформлял из дома, или если покупки оффлайновые, но требуют использования приложения (показать кассиру виртуальную карту) - можно выяснить в каких магазинах клиент предпочитает их делать). Можно добавить в БД пометки "любимый магазин" или "любимое место" (если речь идёт о такси - можно обогатить базу любимыми точками, из которых человек вызывает такси, и в какие ездит)*